<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security
This project is licensed under the MIT License.
-->

*Copyright (c) 2025 Milin Patel. All Rights Reserved.*

# Hazard Analysis and Risk Assessment (HARA)

**Module 03: Functional Safety - ISO 26262**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/03_Functional_Safety/notebooks/02_hara_methodology.ipynb)

**Author:** Milin Patel  
**Institution:** Hochschule Kempten - University of Applied Sciences

---

## Learning Objectives

By the end of this notebook, you will:
- Understand the HARA process as defined in ISO 26262
- Learn to identify hazards systematically for perception systems
- Master the classification of Severity (S), Exposure (E), and Controllability (C)
- Determine ASIL levels from hazard classifications
- Derive safety goals from identified hazards
- Apply HARA to autonomous driving perception systems

---

## 1. Introduction to HARA

**Hazard Analysis and Risk Assessment (HARA)** is a systematic method defined in ISO 26262 Part 3 to identify and evaluate potential hazards arising from malfunctioning behavior of automotive E/E systems.

### Purpose of HARA

- Identify hazardous events that could result from item malfunctions
- Classify the risk level of each hazardous event
- Determine the Automotive Safety Integrity Level (ASIL)
- Define safety goals to prevent or mitigate hazards

### HARA in the Safety Lifecycle

HARA is performed during the **Concept Phase** (ISO 26262 Part 3):

```
Item Definition → HARA → Functional Safety Concept → Technical Safety Requirements
```

The outputs of HARA drive all subsequent safety activities.

In [ ]:
# Setup: Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Setup complete. Libraries imported.")

## 2. HARA Process Overview

The HARA process consists of the following steps:

1. **Item Definition** - Define the system under analysis
2. **Situation Analysis** - Identify operational situations and driving scenarios
3. **Hazard Identification** - Identify potential malfunctions and their consequences
4. **Hazard Classification** - Classify S, E, C for each hazard
5. **ASIL Determination** - Calculate ASIL from S, E, C
6. **Safety Goal Definition** - Define safety goals for hazards with ASIL A-D

In [ ]:
def visualize_hara_process():
    """Visualize the HARA process flow."""
    fig, ax = plt.subplots(figsize=(14, 6))
    
    steps = [
        ('Item\nDefinition', '#3498db'),
        ('Situation\nAnalysis', '#2ecc71'),
        ('Hazard\nIdentification', '#e74c3c'),
        ('Hazard\nClassification\n(S, E, C)', '#9b59b6'),
        ('ASIL\nDetermination', '#f39c12'),
        ('Safety Goal\nDefinition', '#1abc9c')
    ]
    
    x_positions = np.linspace(0.5, 5.5, len(steps))
    
    for i, (label, color) in enumerate(steps):
        box = FancyBboxPatch((x_positions[i]-0.35, 0.3), 0.7, 0.4,
                            boxstyle="round,pad=0.05",
                            facecolor=color, edgecolor='black',
                            linewidth=2, alpha=0.8)
        ax.add_patch(box)
        ax.text(x_positions[i], 0.5, label, ha='center', va='center',
               fontsize=10, fontweight='bold', color='white')
        
        if i < len(steps) - 1:
            ax.annotate('', xy=(x_positions[i+1]-0.4, 0.5),
                       xytext=(x_positions[i]+0.4, 0.5),
                       arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    ax.set_xlim(0, 6)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('HARA Process Flow (ISO 26262 Part 3)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_hara_process()

## 3. Step 1: Item Definition

Before conducting HARA, we must clearly define the **item** (system or function) under analysis.

### Item Definition Template

| Attribute | Description |
|-----------|-------------|
| **Item Name** | Name of the system/function |
| **Description** | What the item does |
| **Boundaries** | What is included/excluded |
| **Interfaces** | Inputs, outputs, interactions |
| **Operating Modes** | Normal, degraded, failure modes |
| **Environmental Conditions** | Weather, lighting, road types |

### Example: Pedestrian Detection System

In [ ]:
# Define item for analysis
item_definition = {
    'Item Name': 'Pedestrian Detection System',
    'Description': 'Camera and LiDAR-based system that detects pedestrians '
                   'in the vehicle path and provides input to AEB system',
    'Boundaries': [
        'Includes: Camera sensors, LiDAR sensors, perception algorithms',
        'Includes: Object classification and tracking',
        'Excludes: Braking actuation (separate item)',
        'Excludes: Driver warning HMI'
    ],
    'Interfaces': {
        'Inputs': ['Camera image stream', 'LiDAR point cloud', 'Vehicle speed'],
        'Outputs': ['Pedestrian detection list', 'Confidence scores', 'Time-to-collision']
    },
    'Operating Modes': ['Normal operation', 'Degraded (sensor failure)', 'Initialization'],
    'Environmental Conditions': ['Day/Night', 'Clear/Rain/Fog', 'Urban/Highway']
}

print("Item Definition: Pedestrian Detection System")
print("=" * 50)
for key, value in item_definition.items():
    if isinstance(value, list):
        print(f"\n{key}:")
        for v in value:
            print(f"  - {v}")
    elif isinstance(value, dict):
        print(f"\n{key}:")
        for k, v in value.items():
            print(f"  {k}: {v}")
    else:
        print(f"\n{key}: {value}")

## 4. Step 2: Situation Analysis

Identify **operational situations** where the item is used. Consider:

- Road types (highway, urban, rural)
- Traffic conditions (dense, sparse, stationary)
- Environmental conditions (weather, lighting)
- Vehicle states (speed, maneuvers)

### Operational Situation Categories (ISO 26262)

In [ ]:
# Define operational situations
operational_situations = pd.DataFrame({
    'ID': ['OS-01', 'OS-02', 'OS-03', 'OS-04', 'OS-05', 'OS-06', 'OS-07', 'OS-08'],
    'Road Type': ['Urban', 'Urban', 'Highway', 'Highway', 'Rural', 'Parking', 'Urban', 'Highway'],
    'Speed Range': ['0-50 km/h', '30-60 km/h', '80-130 km/h', '0-30 km/h', '50-100 km/h', 
                   '0-20 km/h', '0-50 km/h', '100-130 km/h'],
    'Traffic': ['Dense', 'Moderate', 'Sparse', 'Traffic jam', 'Sparse', 'Pedestrians', 
               'School zone', 'Free flow'],
    'Environment': ['Day, clear', 'Night, clear', 'Day, rain', 'Night, fog', 'Dusk', 
                   'Underground', 'Day, clear', 'Day, glare'],
    'Pedestrian Likelihood': ['High', 'Medium', 'Low', 'Low', 'Low', 'High', 'Very High', 'Very Low']
})

print("Operational Situations for Pedestrian Detection System")
display(operational_situations)

## 5. Step 3: Hazard Identification

For each operational situation, identify potential **malfunctions** and their resulting **hazards**.

### Malfunction Types for Perception Systems

| Malfunction Type | Description | Example |
|------------------|-------------|--------|
| **False Negative** | Object present but not detected | Pedestrian not detected |
| **False Positive** | Object detected but not present | Ghost detection causes braking |
| **Late Detection** | Object detected too late | Insufficient time to brake |
| **Misclassification** | Object detected but wrong class | Pedestrian classified as post |
| **Position Error** | Object detected at wrong location | Incorrect distance estimate |
| **Tracking Loss** | Object tracking interrupted | Lost track during occlusion |

In [ ]:
# Define hazards with malfunctions
hazards = pd.DataFrame({
    'Hazard ID': ['H-001', 'H-002', 'H-003', 'H-004', 'H-005', 'H-006'],
    'Malfunction': [
        'False negative (pedestrian not detected)',
        'Late detection (insufficient reaction time)',
        'False positive (phantom detection)',
        'Misclassification (pedestrian as static object)',
        'Position error (incorrect distance)',
        'Tracking loss during occlusion'
    ],
    'Hazardous Event': [
        'Collision with undetected pedestrian',
        'Collision due to late braking',
        'Unnecessary hard braking, rear collision',
        'Collision with misclassified pedestrian',
        'Incorrect braking force, collision or discomfort',
        'Collision with previously tracked pedestrian'
    ],
    'Potential Harm': [
        'Fatal/severe injury to pedestrian',
        'Severe injury to pedestrian',
        'Whiplash injury to occupants',
        'Fatal/severe injury to pedestrian',
        'Injury severity varies with error magnitude',
        'Fatal/severe injury to pedestrian'
    ]
})

print("Identified Hazards for Pedestrian Detection System")
display(hazards)

## 6. Step 4: Hazard Classification (S, E, C)

Each hazard is classified according to three parameters:

### Severity (S) - Extent of potential harm

| Class | Description | Examples |
|-------|-------------|----------|
| **S0** | No injuries | Cosmetic damage only |
| **S1** | Light to moderate injuries | Whiplash, minor cuts |
| **S2** | Severe injuries, survival probable | Broken bones, serious lacerations |
| **S3** | Life-threatening, survival uncertain | Fatal injuries possible |

### Exposure (E) - Probability of operational situation

| Class | Probability | Description |
|-------|-------------|-------------|
| **E0** | Incredible | < 10^-9 per driving hour |
| **E1** | Very low | Rare situations |
| **E2** | Low | Few times per year |
| **E3** | Medium | Once per month |
| **E4** | High | Nearly every drive |

### Controllability (C) - Ability to avoid harm

| Class | Description | Driver capability |
|-------|-------------|------------------|
| **C0** | Generally controllable | > 99% can avoid |
| **C1** | Simply controllable | > 99% can avoid |
| **C2** | Normally controllable | > 90% can avoid |
| **C3** | Difficult or uncontrollable | < 90% can avoid |

In [ ]:
def visualize_sec_scales():
    """Visualize S, E, C classification scales."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Severity
    severity_data = ['S0\nNo injury', 'S1\nLight', 'S2\nSevere', 'S3\nFatal']
    colors_s = ['#27ae60', '#f1c40f', '#e67e22', '#e74c3c']
    axes[0].barh(range(4), [1, 2, 3, 4], color=colors_s, edgecolor='black')
    axes[0].set_yticks(range(4))
    axes[0].set_yticklabels(severity_data)
    axes[0].set_xlabel('Risk Level')
    axes[0].set_title('Severity (S)', fontweight='bold')
    axes[0].invert_yaxis()
    
    # Exposure
    exposure_data = ['E0\nIncredible', 'E1\nVery Low', 'E2\nLow', 'E3\nMedium', 'E4\nHigh']
    colors_e = ['#ecf0f1', '#bdc3c7', '#95a5a6', '#7f8c8d', '#2c3e50']
    axes[1].barh(range(5), [1, 2, 3, 4, 5], color=colors_e, edgecolor='black')
    axes[1].set_yticks(range(5))
    axes[1].set_yticklabels(exposure_data)
    axes[1].set_xlabel('Probability')
    axes[1].set_title('Exposure (E)', fontweight='bold')
    axes[1].invert_yaxis()
    
    # Controllability
    control_data = ['C0\nControllable', 'C1\nSimply', 'C2\nNormally', 'C3\nDifficult']
    colors_c = ['#27ae60', '#2ecc71', '#f39c12', '#c0392b']
    axes[2].barh(range(4), [1, 2, 3, 4], color=colors_c, edgecolor='black')
    axes[2].set_yticks(range(4))
    axes[2].set_yticklabels(control_data)
    axes[2].set_xlabel('Difficulty Level')
    axes[2].set_title('Controllability (C)', fontweight='bold')
    axes[2].invert_yaxis()
    
    plt.suptitle('ISO 26262 Hazard Classification Parameters', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

visualize_sec_scales()

## 7. Step 5: ASIL Determination

The ASIL is determined by combining S, E, and C classifications using the ISO 26262 ASIL determination table.

### ASIL Levels

- **QM** (Quality Management): No safety requirements beyond standard quality
- **ASIL A**: Lowest safety requirements
- **ASIL B**: Moderate safety requirements
- **ASIL C**: High safety requirements
- **ASIL D**: Highest safety requirements

In [ ]:
# ASIL Determination Matrix
ASIL_MATRIX = {
    'S1': {
        'E1': {'C1': 'QM', 'C2': 'QM', 'C3': 'QM'},
        'E2': {'C1': 'QM', 'C2': 'QM', 'C3': 'QM'},
        'E3': {'C1': 'QM', 'C2': 'QM', 'C3': 'A'},
        'E4': {'C1': 'QM', 'C2': 'A', 'C3': 'B'}
    },
    'S2': {
        'E1': {'C1': 'QM', 'C2': 'QM', 'C3': 'QM'},
        'E2': {'C1': 'QM', 'C2': 'QM', 'C3': 'A'},
        'E3': {'C1': 'QM', 'C2': 'A', 'C3': 'B'},
        'E4': {'C1': 'A', 'C2': 'B', 'C3': 'C'}
    },
    'S3': {
        'E1': {'C1': 'QM', 'C2': 'QM', 'C3': 'A'},
        'E2': {'C1': 'QM', 'C2': 'A', 'C3': 'B'},
        'E3': {'C1': 'A', 'C2': 'B', 'C3': 'C'},
        'E4': {'C1': 'B', 'C2': 'C', 'C3': 'D'}
    }
}

def determine_asil(severity: str, exposure: str, controllability: str) -> str:
    """Determine ASIL level based on S, E, C parameters."""
    if severity == 'S0' or exposure == 'E0' or controllability == 'C0':
        return 'QM'
    try:
        return ASIL_MATRIX[severity][exposure][controllability]
    except KeyError:
        return 'QM'

def visualize_asil_matrix():
    """Visualize the complete ASIL determination matrix."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    asil_colors = {
        'QM': '#95a5a6', 'A': '#f1c40f', 'B': '#e67e22', 
        'C': '#e74c3c', 'D': '#8e44ad'
    }
    asil_values = {'QM': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4}
    
    for idx, sev in enumerate(['S1', 'S2', 'S3']):
        matrix = np.zeros((4, 3))
        labels = []
        
        for i, exp in enumerate(['E1', 'E2', 'E3', 'E4']):
            row = []
            for j, ctrl in enumerate(['C1', 'C2', 'C3']):
                asil = ASIL_MATRIX[sev][exp][ctrl]
                matrix[i, j] = asil_values[asil]
                row.append(asil)
            labels.append(row)
        
        im = axes[idx].imshow(matrix, cmap='YlOrRd', aspect='auto', vmin=0, vmax=4)
        
        for i in range(4):
            for j in range(3):
                color = 'white' if matrix[i,j] > 2 else 'black'
                axes[idx].text(j, i, labels[i][j], ha='center', va='center',
                             fontsize=14, fontweight='bold', color=color)
        
        axes[idx].set_xticks([0, 1, 2])
        axes[idx].set_xticklabels(['C1', 'C2', 'C3'])
        axes[idx].set_yticks([0, 1, 2, 3])
        axes[idx].set_yticklabels(['E1', 'E2', 'E3', 'E4'])
        axes[idx].set_xlabel('Controllability', fontweight='bold')
        axes[idx].set_ylabel('Exposure', fontweight='bold')
        axes[idx].set_title(f'Severity = {sev}', fontsize=12, fontweight='bold')
    
    plt.suptitle('ASIL Determination Matrix (ISO 26262)', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

visualize_asil_matrix()

## 8. Complete HARA Example

Let's perform a complete HARA for the Pedestrian Detection System, classifying each hazard and determining its ASIL.

In [ ]:
# Complete HARA table
hara_complete = pd.DataFrame({
    'Hazard ID': ['H-001', 'H-002', 'H-003', 'H-004', 'H-005', 'H-006'],
    'Malfunction': [
        'False negative',
        'Late detection',
        'False positive',
        'Misclassification',
        'Position error',
        'Tracking loss'
    ],
    'Operational Situation': [
        'Urban, 50 km/h, pedestrian crossing',
        'Highway exit, 80 km/h',
        'Highway, 120 km/h, following traffic',
        'Urban, 40 km/h, pedestrian near road',
        'Any speed, pedestrian in path',
        'Urban, 30 km/h, occluded pedestrian'
    ],
    'Severity': ['S3', 'S3', 'S2', 'S3', 'S2', 'S3'],
    'Exposure': ['E4', 'E3', 'E3', 'E4', 'E3', 'E3'],
    'Controllability': ['C3', 'C3', 'C2', 'C3', 'C2', 'C2']
})

# Calculate ASIL for each hazard
hara_complete['ASIL'] = hara_complete.apply(
    lambda row: determine_asil(row['Severity'], row['Exposure'], row['Controllability']),
    axis=1
)

print("Complete HARA for Pedestrian Detection System")
print("=" * 80)
display(hara_complete)

In [ ]:
# Visualize ASIL distribution
def visualize_asil_distribution(hara_df):
    """Visualize ASIL distribution from HARA results."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart by hazard
    asil_order = {'QM': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4}
    colors = {'QM': '#95a5a6', 'A': '#f1c40f', 'B': '#e67e22', 'C': '#e74c3c', 'D': '#8e44ad'}
    
    bar_colors = [colors[asil] for asil in hara_df['ASIL']]
    bars = ax1.bar(hara_df['Hazard ID'], [asil_order[a] for a in hara_df['ASIL']], 
                   color=bar_colors, edgecolor='black', linewidth=2)
    
    ax1.set_ylabel('ASIL Level', fontweight='bold')
    ax1.set_xlabel('Hazard ID', fontweight='bold')
    ax1.set_title('ASIL by Hazard', fontsize=12, fontweight='bold')
    ax1.set_yticks([0, 1, 2, 3, 4])
    ax1.set_yticklabels(['QM', 'A', 'B', 'C', 'D'])
    
    for i, (bar, asil) in enumerate(zip(bars, hara_df['ASIL'])):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                asil, ha='center', fontweight='bold')
    
    # Pie chart of ASIL distribution
    asil_counts = hara_df['ASIL'].value_counts()
    pie_colors = [colors[a] for a in asil_counts.index]
    ax2.pie(asil_counts.values, labels=asil_counts.index, colors=pie_colors,
           autopct='%1.0f%%', startangle=90, explode=[0.05]*len(asil_counts),
           textprops={'fontweight': 'bold'})
    ax2.set_title('ASIL Distribution', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_asil_distribution(hara_complete)

## 9. Step 6: Safety Goal Definition

For each hazard with ASIL A-D, we define a **Safety Goal (SG)**.

### Safety Goal Attributes

| Attribute | Description |
|-----------|-------------|
| **ID** | Unique identifier |
| **Description** | What must be achieved |
| **ASIL** | Inherited from hazard |
| **Safe State** | State to achieve when hazard occurs |
| **Fault Tolerant Time** | Time available to reach safe state |

In [ ]:
# Derive safety goals from hazards
def derive_safety_goals(hara_df):
    """Derive safety goals from HARA results."""
    safety_goals = []
    
    goal_templates = {
        'H-001': ('SG-001', 'The system shall detect all pedestrians in the vehicle path', 
                 'Controlled stop or collision avoidance', '500 ms'),
        'H-002': ('SG-002', 'The system shall detect pedestrians with sufficient time for braking',
                 'Emergency braking activated', '200 ms'),
        'H-003': ('SG-003', 'The system shall not trigger false emergency braking on highways',
                 'No unintended braking', 'N/A'),
        'H-004': ('SG-004', 'The system shall correctly classify pedestrians vs static objects',
                 'Appropriate braking response', '300 ms'),
        'H-005': ('SG-005', 'The system shall provide accurate pedestrian position estimates',
                 'Correct braking force applied', '200 ms'),
        'H-006': ('SG-006', 'The system shall maintain pedestrian tracking during occlusions',
                 'Predictive tracking active', '1000 ms')
    }
    
    for _, row in hara_df.iterrows():
        if row['ASIL'] != 'QM':
            sg_id, description, safe_state, ftti = goal_templates[row['Hazard ID']]
            safety_goals.append({
                'SG ID': sg_id,
                'Hazard ID': row['Hazard ID'],
                'Safety Goal': description,
                'ASIL': row['ASIL'],
                'Safe State': safe_state,
                'FTTI': ftti
            })
    
    return pd.DataFrame(safety_goals)

safety_goals_df = derive_safety_goals(hara_complete)
print("Safety Goals Derived from HARA")
print("=" * 80)
display(safety_goals_df)

## 10. Interactive HARA Tool

Use this tool to perform your own HARA analysis.

In [ ]:
class HARAWorksheet:
    """Interactive HARA worksheet for automotive systems."""
    
    def __init__(self, item_name: str):
        self.item_name = item_name
        self.hazards = []
        self.safety_goals = []
    
    def add_hazard(self, hazard_id: str, malfunction: str, 
                   operational_situation: str, severity: str, 
                   exposure: str, controllability: str):
        """Add a hazard to the HARA."""
        asil = determine_asil(severity, exposure, controllability)
        
        hazard = {
            'Hazard ID': hazard_id,
            'Malfunction': malfunction,
            'Operational Situation': operational_situation,
            'Severity': severity,
            'Exposure': exposure,
            'Controllability': controllability,
            'ASIL': asil
        }
        self.hazards.append(hazard)
        print(f"Added {hazard_id}: {malfunction} -> ASIL {asil}")
        return asil
    
    def add_safety_goal(self, sg_id: str, hazard_id: str, 
                        description: str, safe_state: str):
        """Add a safety goal."""
        # Find ASIL from corresponding hazard
        asil = 'QM'
        for h in self.hazards:
            if h['Hazard ID'] == hazard_id:
                asil = h['ASIL']
                break
        
        self.safety_goals.append({
            'SG ID': sg_id,
            'Hazard ID': hazard_id,
            'Safety Goal': description,
            'ASIL': asil,
            'Safe State': safe_state
        })
    
    def get_hazards_df(self):
        return pd.DataFrame(self.hazards)
    
    def get_safety_goals_df(self):
        return pd.DataFrame(self.safety_goals)
    
    def summary(self):
        """Print HARA summary."""
        print(f"\nHARA Summary for: {self.item_name}")
        print("=" * 60)
        print(f"Total Hazards: {len(self.hazards)}")
        
        asil_counts = {}
        for h in self.hazards:
            asil_counts[h['ASIL']] = asil_counts.get(h['ASIL'], 0) + 1
        
        print("\nASIL Distribution:")
        for asil in ['D', 'C', 'B', 'A', 'QM']:
            if asil in asil_counts:
                print(f"  ASIL {asil}: {asil_counts[asil]} hazards")
        
        print(f"\nSafety Goals Defined: {len(self.safety_goals)}")

# Example usage
print("HARA Worksheet Tool")
print("=" * 60)
worksheet = HARAWorksheet("Lane Keeping Assist System")

# Add example hazards
worksheet.add_hazard('H-LKA-001', 'Unintended lane departure', 
                     'Highway, 120 km/h, driver distracted', 'S3', 'E3', 'C2')
worksheet.add_hazard('H-LKA-002', 'Steering towards oncoming traffic',
                     'Rural road, 80 km/h, faded lane markings', 'S3', 'E2', 'C3')
worksheet.add_hazard('H-LKA-003', 'False lane detection on construction zone',
                     'Highway, 60 km/h, temporary markings', 'S2', 'E3', 'C2')

worksheet.summary()

## 11. HARA Best Practices

### Common Mistakes to Avoid

1. **Confusing hazard with malfunction** - Hazard is the consequence, malfunction is the cause
2. **Inconsistent S/E/C classification** - Use consistent criteria across hazards
3. **Missing operational situations** - Consider all relevant driving scenarios
4. **Over-conservative ratings** - Be realistic, not pessimistic
5. **Ignoring controllability by others** - Consider pedestrians, other drivers

### Documentation Requirements

ISO 26262 requires documentation of:
- Item definition
- All identified hazards with classification rationale
- ASIL determination for each hazard
- Safety goals with safe states
- Traceability between hazards and safety goals

### References

- ISO 26262:2018 Part 3 - Concept Phase
- SAE J2980 - Considerations for ISO 26262 ASIL Hazard Classification
- Patel, M., Jung, R., Khatun, M. (2024). "Assessing Unknown Hazards for SOTIF Based on Twin Scenarios." IEEE.

## 12. Exercise: Perform Your Own HARA

**Task:** Perform a HARA for an **Adaptive Cruise Control (ACC)** system.

Consider these malfunctions:
1. Failure to detect lead vehicle
2. False detection causing unnecessary braking
3. Incorrect distance estimation
4. Late response to cut-in vehicle

For each malfunction:
- Identify operational situations
- Classify S, E, C
- Determine ASIL
- Define safety goals

In [ ]:
# Exercise: Your HARA for ACC
# Create your worksheet and add hazards

acc_hara = HARAWorksheet("Adaptive Cruise Control")

# TODO: Add your hazards here
# Example:
# acc_hara.add_hazard('H-ACC-001', 'Failure to detect lead vehicle',
#                     'Highway, 130 km/h, following traffic', 'S?', 'E?', 'C?')

# Uncomment and complete:
# acc_hara.add_hazard('H-ACC-001', ..., ..., 'S3', 'E4', 'C2')
# acc_hara.add_hazard('H-ACC-002', ..., ..., 'S?', 'E?', 'C?')
# acc_hara.add_hazard('H-ACC-003', ..., ..., 'S?', 'E?', 'C?')
# acc_hara.add_hazard('H-ACC-004', ..., ..., 'S?', 'E?', 'C?')

# acc_hara.summary()
# display(acc_hara.get_hazards_df())

## Summary

In this notebook, you learned:

- **HARA Process**: Systematic method for identifying and classifying automotive hazards
- **S, E, C Classification**: How to rate severity, exposure, and controllability
- **ASIL Determination**: How to derive ASIL from hazard classifications
- **Safety Goals**: How to define top-level safety requirements from hazards
- **Practical Application**: HARA for perception systems in autonomous vehicles

### Next Steps

- **Notebook 03**: ASIL Decomposition and Requirements Allocation
- **Module 04**: SOTIF for performance limitations (complements HARA)
- **Module 05**: TARA for cybersecurity threats

---

*Notebook created by Milin Patel | Hochschule Kempten*  
*Last updated: 2025-01-22*